In [1]:
# i cant see any task except home work from .pdf file - 'Get practice', and i make some evaluation of random models

In [6]:
import platform
print(platform.python_version())

3.11.0rc1


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import pytorch_lightning as pl

from torch import nn
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [27]:
model_name = 'mnist_model_64.pt'
device = 'cuda'
batch_size = 64
precision_model = 64 #'bf16-mixed'
num_workers = 15
epochs = 20

In [23]:
class LitMNIST(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 256)
        self.layer_3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        return self.layer_3(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)


class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=64, num_workers=8):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.transform = transforms.ToTensor()

    def prepare_data(self):
        MNIST(root=".", train=True, download=True)
        MNIST(root=".", train=False, download=True)

    def setup(self, stage=None):
        mnist_full = MNIST(root=".", train=True, transform=self.transform)
        self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])
        self.mnist_test = MNIST(root=".", train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers)

In [3]:
help(pl.Trainer)

Help on class Trainer in module pytorch_lightning.trainer.trainer:

class Trainer(builtins.object)
 |  Trainer(*, accelerator: Union[str, pytorch_lightning.accelerators.accelerator.Accelerator] = 'auto', strategy: Union[str, pytorch_lightning.strategies.strategy.Strategy] = 'auto', devices: Union[list[int], str, int] = 'auto', num_nodes: int = 1, precision: Union[Literal[64, 32, 16], Literal['transformer-engine', 'transformer-engine-float16', '16-true', '16-mixed', 'bf16-true', 'bf16-mixed', '32-true', '64-true'], Literal['64', '32', '16', 'bf16'], NoneType] = None, logger: Union[pytorch_lightning.loggers.logger.Logger, collections.abc.Iterable[pytorch_lightning.loggers.logger.Logger], bool, NoneType] = None, callbacks: Union[list[pytorch_lightning.callbacks.callback.Callback], pytorch_lightning.callbacks.callback.Callback, NoneType] = None, fast_dev_run: Union[int, bool] = False, max_epochs: Optional[int] = None, min_epochs: Optional[int] = None, max_steps: int = -1, min_steps: Option

In [28]:
model = LitMNIST()
dm = MNISTDataModule(batch_size=batch_size, num_workers=num_workers)

trainer = pl.Trainer(
    max_epochs=epochs, 
    accelerator='gpu' if torch.cuda.is_available() and device == 'cuda' else 'cpu',
    precision=precision_model,
    detect_anomaly=True,
    accumulate_grad_batches=batch_size
)
trainer.fit(model, datamodule=dm)

torch.save(model.state_dict(), model_name)
print(f"Model saved to {model_name}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/forever/.local/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params | Mode 
-------------------------------------------
0 | layer_1 | Linear | 100 K  | train
1 | layer_2 | Linear | 33.0 K | train
2 | layer_3 | Linear | 2.6 K  | train
-------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)
3   

Epoch 19: 100%|████████████████████████████████████████████████████████| 860/860 [00:09<00:00, 89.22it/s, v_num=8]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|████████████████████████████████████████████████████████| 860/860 [00:09<00:00, 89.10it/s, v_num=8]
Model saved to mnist_model_64.pt


In [29]:
model = LitMNIST()
model.load_state_dict(torch.load(model_name))
model.to(device)
model.eval()

LitMNIST(
  (layer_1): Linear(in_features=784, out_features=128, bias=True)
  (layer_2): Linear(in_features=128, out_features=256, bias=True)
  (layer_3): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
# precision bf16-mixed

In [26]:
transform = transforms.ToTensor()
test_dataset = MNIST(root=".", train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        outputs = model(x)
        preds = torch.argmax(outputs, dim=1)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.9619

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.98      0.98      0.98      1135
           2       0.97      0.96      0.96      1032
           3       0.96      0.96      0.96      1010
           4       0.95      0.97      0.96       982
           5       0.96      0.95      0.95       892
           6       0.95      0.97      0.96       958
           7       0.96      0.96      0.96      1028
           8       0.95      0.95      0.95       974
           9       0.96      0.94      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000


Confusion Matrix:
 [[ 963    0    1    2    1    3    7    2    1    0]
 [   0 1117    4    0    1    1    5    2    5    0]
 [   4    2  991    3    5    1    6    6   12    2]
 [   0    0    7  96

In [ ]:
# precision 64

In [30]:
transform = transforms.ToTensor()
test_dataset = MNIST(root=".", train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        outputs = model(x)
        preds = torch.argmax(outputs, dim=1)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.96

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.96      0.96      0.96      1032
           3       0.95      0.96      0.95      1010
           4       0.95      0.97      0.96       982
           5       0.95      0.95      0.95       892
           6       0.97      0.95      0.96       958
           7       0.96      0.96      0.96      1028
           8       0.96      0.94      0.95       974
           9       0.96      0.94      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000


Confusion Matrix:
 [[ 960    0    2    1    0    6    5    3    1    2]
 [   0 1122    2    1    0    1    3    2    4    0]
 [   4    3  990    6    4    2    6    9    7    1]
 [   0    0    9  970 

In [44]:
model = LitMNIST()
model.load_state_dict(torch.load('mnist_model.pt'))
model.eval()
quantizer = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
torch.save(quantizer.state_dict(), 'quant.pt')
# quantizer.to(device)

In [45]:
transform = transforms.ToTensor()
test_dataset = MNIST(root=".", train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

y_true = []
y_pred = []

with torch.no_grad():
    for x, y in test_loader:
        # x = x.to(device)
        outputs = quantizer(x)
        preds = torch.argmax(outputs, dim=1)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.9617

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       980
           1       0.98      0.98      0.98      1135
           2       0.97      0.96      0.96      1032
           3       0.96      0.96      0.96      1010
           4       0.95      0.97      0.96       982
           5       0.96      0.95      0.95       892
           6       0.95      0.97      0.96       958
           7       0.97      0.96      0.96      1028
           8       0.95      0.95      0.95       974
           9       0.96      0.94      0.95      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000


Confusion Matrix:
 [[ 963    0    1    2    1    3    7    2    1    0]
 [   0 1117    4    0    1    1    5    2    5    0]
 [   4    2  991    3    5    1    6    6   12    2]
 [   0    0    7  96